## Recommender System 

In [1]:
import pandas as pd
import numpy as np
import ast
import warnings
warnings.simplefilter('default')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter  import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

## Fetching and Converting to DataFrame

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

## Sample movies

In [3]:
movies.sample(10)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
723,60000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 878, ""...",http://www.thehappeningmovie.com,8645,"[{""id"": 3352, ""name"": ""tree""}, {""id"": 5096, ""n...",en,The Happening,When a deadly airborne virus threatens to wipe...,20.501582,"[{""name"": ""Spyglass Entertainment"", ""id"": 158}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2008-06-11,163403799,91.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,We've Sensed It. We've Seen The Signs. Now... ...,The Happening,4.9,950
4797,0,"[{""id"": 10769, ""name"": ""Foreign""}, {""id"": 53, ...",NaN,67238,[],en,Cavite,"Adam, a security guard, travels from Californi...",0.022173,[],[],2005-03-12,0,80.0,[],Released,NaN,Cavite,7.5,2
4062,2100000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",NaN,887,"[{""id"": 279, ""name"": ""usa""}, {""id"": 447, ""name...",en,The Best Years of Our Lives,It's the hope that sustains the spirit of ever...,8.387170,"[{""name"": ""Samuel Goldwyn Company, The"", ""id"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1946-12-25,23650000,172.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Three wonderful loves in the best picture of t...,The Best Years of Our Lives,7.6,143
3385,0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,25388,"[{""id"": 394, ""name"": ""gypsy""}, {""id"": 539, ""na...",en,March or Die,"The French Foreign Legion, in the early 20s, i...",3.829937,"[{""name"": ""Associated General Films"", ""id"": 43...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",1977-09-08,0,107.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,March or Die,6.2,16
694,60000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 18, ""nam...",http://www.universalstudiosentertainment.com/t...,2501,"[{""id"": 90, ""name"": ""paris""}, {""id"": 107, ""nam...",en,The Bourne Identity,Wounded to the brink of death and suffering fr...,86.476817,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2002-06-14,214034224,119.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,He was the perfect weapon until he became the ...,The Bourne Identity,7.3,3583
853,55000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",NaN,4517,"[{""id"": 392, ""name"": ""england""}, {""id"": 441, ""...",en,Elizabeth: The Golden Age,When Queen Elizabeth's reign is threatened by ...,15.274137,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2007-09-09,74237563,114.0,"[{""iso_639_1"": ""sv"", ""name"": ""svenska""}, {""iso...",Released,Woman. Warrior. Queen.,Elizabeth: The Golden Age,6.6,303
1261,0,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,11025,"[{""id"": 1608, ""name"": ""twin sister""}, {""id"": 5...",en,New York Minute,Top student Jane Ryan heads to Manhattan for a...,10.040300,"[{""name"": ""DiNovi Pictures"", ""id"": 813}, {""nam...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2004-05-01,0,91.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,New York Minute,5.5,185
3018,10000000,"[{""id"": 9648, ""name"": ""Mystery""}, {""id"": 878, ...",http://www.thefourthkind.net/,22824,"[{""id"": 1155, ""name"": ""brother sister relation...",en,The Fourth Kind,"Since the 1960s, a disproportionate number of ...",22.888173,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-11-06,42333295,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There are four kinds of alien encounters. The ...,The Fourth Kind,5.8,441
2645,12000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28

In [4]:
credits.sample(10)

,movie_id,title,cast,crew
2910,358451,A Tale of Three Cities,"[{""cast_id"": 0, ""character"": ""Fang Daolong"", ""...","[{""credit_id"": ""57e7b357925141390b01049a"", ""de..."
4667,77332,Urbania,"[{""cast_id"": 2, ""character"": ""Charlie"", ""credi...","[{""credit_id"": ""52fe4967c3a368484e129997"", ""de..."
55,62177,Brave,"[{""cast_id"": 1, ""character"": ""M\u00e9rida (voi...","[{""credit_id"": ""54a3ff9f9251414d2700c759"", ""de..."
36,91314,Transformers: Age of Extinction,"[{""cast_id"": 6, ""character"": ""Cade Yeager"", ""c...","[{""credit_id"": ""546c7de8c3a368096b001181"", ""de..."
4086,20862,Smoke Signals,"[{""cast_id"": 2, ""character"": """", ""credit_id"": ...","[{""credit_id"": ""52fe43fec3a368484e009967"", ""de..."
1082,2207,16 Blocks,"[{""cast_id"": 14, ""character"": ""Jack Mosley"", ""...","[{""credit_id"": ""52fe4341c3a36847f804620f"", ""de..."
399,7484,Open Season,"[{""cast_id"": 7, ""character"": ""Boog (voice)"", ""...","[{""credit_id"": ""52fe447fc3a36847f8099b9b"", ""de..."
3766,5,Four Rooms,"[{""cast_id"": 42, ""character"": ""Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800012d"", ""de..."
1692,24071,Mumford,"[{""cast_id"": 3, ""character"": ""Mumford"", ""credi...","[{""credit_id"": ""52fe4480c3a368484e026c51"", ""de..."
4789,39851,Clean,"[{""cast_id"": 1, ""character"": ""Emily Wang"", ""cr...","[{""credit_id"": ""52fe47369251416c9106db9b"", ""de..."


## Merging the Two  DataFrames On the Title Column

In [5]:
movies=movies.merge(credits,on='title')

In [6]:
movies.sample(10)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
4142,1000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 27, ""n...",http://www.grave-encounters.tv/,50698,"[{""id"": 2249, ""name"": ""camcorder""}, {""id"": 302...",en,Grave Encounters,A crew from a paranormal reality television sh...,13.222268,"[{""name"": ""Darclight"", ""id"": 6448}, {""name"": ""...",...,92.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,They Were Searching For Proof. They Found it.,Grave Encounters,6.1,381,50698,"[{""cast_id"": 3, ""character"": ""Lance"", ""credit_...","[{""credit_id"": ""52fe47d3c3a36847f814a2ef"", ""de..."
1384,34000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 16, ""...",https://www.warnerbros.com/tmnt,1273,"[{""id"": 242, ""name"": ""new york""}, {""id"": 380, ...",en,TMNT,"After the defeat of their old arch nemesis, Th...",18.392170,"[{""name"": ""The Weinstein Company"", ""id"": 308},...",...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,TMNT,6.0,345,1273,"[{""cast_id"": 8, ""character"": ""Casey Jones (voi...","[{""credit_id"": ""52fe42ecc3a36847f802d3db"", ""de..."
1185,20000000,"[{""id"": 35, ""name"": ""Comedy""}]",http://www.17againmovie.com/,16996,"[{""id"": 6270, ""name"": ""high school""}, {""id"": 6...",en,17 Again,"On the brink of a midlife crisis, 30-something...",52.571674,"[{""name"": ""New Line Cinema"", ""id"": 12}, {""name...",...,102.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Who says you're only young once?,17 Again,6.1,1350,16996,"[{""cast_id"": 1, ""character"": ""Mike O'Donnell (...","[{""credit_id"": ""563e1c4b9251417fa90036b7"", ""de..."
4413,1000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 35, ""name...",NaN,242083,"[{""id"": 2276, ""name"": ""talent""}, {""id"": 6262, ...",en,Hits,A talentless teen will do anything to get on T...,3.638411,"[{""name"": ""Honora Productions"", ""id"": 39077}, ...",...,96.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Based on a true story... that hasn't happened ...,Hits,4.9,23,242083,"[{""cast_id"": 13, ""character"": ""Katelyn"", ""cred...","[{""credit_id"": ""52fe4ec5c3a36847f82a6cb1"", ""de..."
2200,20000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",http://www.lockoutfilm.com/,81796,"[{""id"": 840, ""name"": ""usa president""}, {""id"": ...",en,Lockout,"Set in the near future, Lockout follows a fals...",35.232278,"[{""name"": ""Canal+"", ""id"": 5358}, {""name"": ""Eur...",...,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Take no prisoners.,Lockout,5.8,715,81796,"[{""cast_id"": 1, ""character"": ""Snow"", ""credit_i...","[{""credit_id"": ""52fe482c9251416c91082d25"", ""de..."
1855,25000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 28, ""n...",http://www.contrabandmovie.net,77866,"[{""id"": 11429, ""name"": ""head wound""}, {""id"": 1...",en,Contraband,When his brother-in-law runs afoul of a drug l...,31.364901,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...",...,109.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,What would you hide to protect your family?,Contraband,6.1,770,77866,"[{""cast_id"": 2, ""character"": ""Chris Farraday"",...","[{""credit_id"": ""52fe497fc3a368484e12e7c7"", ""de..."
3076,11500000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,2088,"[{""id"": 4668, ""name"": ""police operation""}, {""i...",en,Romeo Is Bleeding,A corrupt cop gets in over his head when he tr...,4.850402,"[{""name"": ""Working Title Films"", ""id"": 10163},...",...,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The story of a cop who wanted it bad and got i...,Romeo Is Bleeding,5.7,36,2088,"[{""cast_id"": 11, ""character"": ""Jack Grimaldi"",...","[{""credit_id"": ""52fe4334c3a36847f80422ef"", ""de..."
2116,20000000,"[{""id"": 12, ""name""

## Removing the Unncessary Columns From The DataFrame  


In [7]:
movies=movies[['movie_id','title','overview','genres','keywords','crew','cast']]

In [8]:
movies.head()

,movie_id,title,overview,genres,keywords,crew,cast
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c..."


## Data Preprocessing 

In [9]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
crew        0
cast        0
dtype: int64

In [10]:
movies.dropna(inplace=True)

In [11]:
movies.shape

(4806, 7)

In [12]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

## Extracting genres from the geners Columns

In [13]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [14]:
movies['genres']=movies['genres'].apply(convert)

In [15]:
movies.head()

,movie_id,title,overview,genres,keywords,crew,cast
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c..."


In [16]:
movies['keywords']=movies['keywords'].apply(convert)

In [17]:
movies.head()

,movie_id,title,overview,genres,keywords,crew,cast
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c..."


In [18]:
def actors(object):
    List=[]
    count=0
    for i in ast.literal_eval(object):
        List.append(i['name'])
        count=count+1
        if count==3:
            break
    return List

In [19]:
movies['cast']=movies['cast'].apply(actors)

In [20]:
movies.head()

,movie_id,title,overview,genres,keywords,crew,cast
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[Johnny Depp, Orlando Bloom, Keira Knightley]"
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[Daniel Craig, Christoph Waltz, Léa Seydoux]"
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[Christian Bale, Michael Caine, Gary Oldman]"
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[Taylor Kitsch, Lynn Collins, Samantha Morton]"


In [21]:
def Director_name(parameter):
    List1=[]
    
    
    for i in ast.literal_eval(parameter):
        if i['job']=='Director':
            
            List1.append(i['name'])
    
    return List1
    

In [22]:
movies['crew']=movies['crew'].apply(Director_name)

In [23]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [24]:
movies.head()

,movie_id,title,overview,genres,keywords,crew,cast
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",[James Cameron],"[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",[Gore Verbinski],"[Johnny Depp, Orlando Bloom, Keira Knightley]"
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",[Sam Mendes],"[Daniel Craig, Christoph Waltz, Léa Seydoux]"
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",[Christopher Nolan],"[Christian Bale, Michael Caine, Gary Oldman]"
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",[Andrew Stanton],"[Taylor Kitsch, Lynn Collins, Samantha Morton]"


In [25]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [26]:
movies.head()

,movie_id,title,overview,genres,keywords,crew,cast
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...",[JamesCameron],"[SamWorthington, ZoeSaldana, SigourneyWeaver]"
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...",[GoreVerbinski],"[JohnnyDepp, OrlandoBloom, KeiraKnightley]"
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",[SamMendes],"[DanielCraig, ChristophWaltz, LéaSeydoux]"
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...",[ChristopherNolan],"[ChristianBale, MichaelCaine, GaryOldman]"
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...",[AndrewStanton],"[TaylorKitsch, LynnCollins, SamanthaMorton]"


## Concatinating All The columns into single columns

In [27]:
movies['tages']=movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [28]:
df=movies[['movie_id','title','tages']]

In [29]:
df.head()

,movie_id,title,tages
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [30]:
df['tages']=df['tages'].apply(lambda x:" ".join(x))

C:\Users\win10\AppData\Local\Temp\ipykernel_6296\107550294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tages']=df['tages'].apply(lambda x:" ".join(x))


In [31]:
df.head()

,movie_id,title,tages
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [32]:
df['tages']=df['tages'].apply(lambda x:x.lower())

C:\Users\win10\AppData\Local\Temp\ipykernel_6296\2812894028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tages']=df['tages'].apply(lambda x:x.lower())


In [33]:
ps=PorterStemmer()

In [34]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [35]:
df['tages']=df["tages"].apply(stem)

C:\Users\win10\AppData\Local\Temp\ipykernel_6296\102541313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tages']=df["tages"].apply(stem)


In [36]:
df.head()

,movie_id,title,tages
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


## Calculating Similarity between tags by cosin Similarity to do this we need to convert the tages in vectors 

**1.remove stopwords**

In [37]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [38]:
vectors=cv.fit_transform(df['tages']).toarray()

In [39]:
similarity=cosine_similarity(vectors)

In [40]:
def recomandation(movie):
    movie_index=df[df['title']== movie].index[0] ## to get the index of the movies
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True ,key=lambda x:x[1])[1:6]
    

    for i in movies_list:
        print(df.iloc[i[0]].title)
    
    
    

In [41]:
import pickle

In [42]:
pickle.dump(df.to_dict(),open('movies_dict.pkl','wb'))

C:\Users\win10\AppData\Local\Temp\ipykernel_6296\2252860710.py:1: ResourceWarning: unclosed file <_io.BufferedWriter name='movies_dict.pkl'>
  pickle.dump(df.to_dict(),open('movies_dict.pkl','wb'))


In [43]:
pickle.dump(similarity,open('similarity.pkl','wb'))

C:\Users\win10\AppData\Local\Temp\ipykernel_6296\3696703621.py:1: ResourceWarning: unclosed file <_io.BufferedWriter name='similarity.pkl'>
  pickle.dump(similarity,open('similarity.pkl','wb'))
